# Дипломный AL project студента Жукова Егора

<p>
    <strong>Тема: Разработка pipline модели для QA</strong>
</p>
<p>
    Описание задачи:
    
    Есть открытый набор данных SBERSQUAD, который содержит данные энцеклопидического толка. Есть отрывок (контекст) статьи, ряд вопросов по тексту отрывка,
    а так же ответы из текста. Необходимо подобрать архитектуру модели, разработать пайлпайн ее обучения, с целью научить модель отвечать на вопросы, находя ответы в контексте.
    
    Преимущества реализации:
    Уникальностью решения является возможность модели извлекать ответы из контекста, который она не видела на обучение. При этом ответы максимально приближенны к контексту, как будто у нас Reader система, но при этом в основном лишены недостатков Generator системы (генерация "бреда"). Это позволяет использовать данный подход в большом спектре бизнес задач.
</p>

<p>
    
    Данные:   
    
    https://huggingface.co/datasets/sberquad
    https://arxiv.org/abs/1912.09723
    
    name	train	validation	test
    plain_text	45328	5036	23936
    
    размер обучающей выборки превышает 45 000 строк
    Context - не уникален
    Question - уникален
    Answer - уникален соответсвует вопросу
</p>
<p>
    Архитектура:
    
    После изучения вопроса остановился на архитектуре T5
    https://arxiv.org/pdf/1910.10683.pdf
    
    Предобученная модель от Sberbank AL
    https://huggingface.co/sberbank-ai/ruT5-base
    
</p>
<p>
    Допущения:
    
    Я не стал предобрабатывать текст. Это поле для экспериментов, но в случае с энциклопедией может потерятся смысл имен и названий.
</p>

In [1]:
# импорты необходимых библиотек
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, get_linear_schedule_with_warmup
from datasets import load_dataset

import torch
from torch import cuda
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch.nn.functional as F

# удобный класс для настроек
from types import SimpleNamespace
# метрика, не совсем подходит для русского языка
from rouge import Rouge

import numpy as np
import pandas as pd
import random
import os

import pickle

from tqdm.notebook import tqdm
from datetime import datetime

import wandb

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tqdm.pandas()
wandb.login()
# собираем параметры в одно место
cfg = SimpleNamespace()
cfg.model_name = "sberbank-ai/ruT5-base"
cfg.batch_size = 4
cfg.epoch = 3
cfg.learning_rate = 1e-5
cfg.source_max_len = 439
cfg.target_max_len = 30
cfg.seed = 2022
cfg.path_of_model = 'MODELS/'
cfg.apex = True
cfg.max_grad_norm = 1.0
cfg.n_accumulate = 1
cfg.num_warmup_steps = 0
cfg.eps = 1e-6
cfg.betas = (0.9, 0.999)
cfg.num_workers = 0

wandb: Currently logged in as: resquilleur (use `wandb login --relogin` to force relogin)


In [3]:
# обеспечиваем воспроизводимость результатов
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=cfg.seed)

# 1. Загрузка и обработка данных 

In [4]:
dataset = load_dataset('sberquad')

Reusing dataset sberquad (C:\Users\bitzh\.cache\huggingface\datasets\sberquad\sberquad\1.0.0\62115d937acf2634cfacbfee10c13a7ee39df3ce345bb45af7088676f9811e77)


  0%|          | 0/3 [00:00<?, ?it/s]

## 1.1. Формирование наборов данных

In [5]:
# функция конвертации данных из формата набора в обычный датафрейм для удобства и читаемости данных (нам не нужны позиции ответов)
def DataPrepareToSQUAD(dataset):
    # train dataframe
    tdf = dataset["train"].to_pandas().drop(['id', 'title'], axis=1)
    tdf['answers'] = [x['text'][0] for x in dataset['train']['answers']]
    # validate dataframe
    vdf = dataset["validation"].to_pandas().drop(['id', 'title'], axis=1)
    vdf['answers'] = [x['text'][0] for x in dataset['validation']['answers']]
    return tdf, vdf

train_data, val_data = DataPrepareToSQUAD(dataset)

In [6]:
# train_data = train_data.sample(500).reset_index(drop=True)
# val_data = val_data.sample(100).reset_index(drop=True)

## 1.2. EDA максимальной длины последовательности

In [6]:
# загрузим токенайзер
tokenizer = T5Tokenizer.from_pretrained(cfg.model_name)

In [8]:
data_len = pd.concat([train_data, val_data], axis=0).reset_index(drop=True)

def get_token_len(x):
    x = tokenizer.encode_plus(x,
                          add_special_tokens=True,
                          max_length=512,
                          return_token_type_ids=False,
                          pad_to_max_length=False,
                          return_attention_mask=False,
                          truncation=True
                          )
    return len(x['input_ids'])

data_len['context_token_len'] = data_len['context'].progress_apply(get_token_len)
data_len['question_token_len'] = data_len['question'].progress_apply(get_token_len)
data_len['qc_token_len'] = data_len['question_token_len'] + data_len['context_token_len']
data_len['answers_token_len'] = data_len['answers'].progress_apply(get_token_len)

cfg.source_max_len = int(data_len['qc_token_len'].describe(percentiles=[0.995])['99.5%'])
print(cfg.source_max_len)

cfg.target_max_len = int(data_len['answers_token_len'].describe(percentiles=[0.995])['99.5%'])
print(cfg.target_max_len)

del data_len

  0%|          | 0/50364 [00:00<?, ?it/s]

  0%|          | 0/50364 [00:00<?, ?it/s]

  0%|          | 0/50364 [00:00<?, ?it/s]

439
30


# 2. Обучение модели

In [8]:
class T5DataSet(Dataset):
    """
    Создание пользовательского набора данных для чтения набора данных и
    загрузки его в загрузчик данных для передачи его во вход
    нейронной сети для тонкой настройки модели
    """

    def __init__(
        self, dataframe: pd.DataFrame,
        tokenizer: T5Tokenizer, question_len: int = 0,
        answers_len: int = 0,
        switch: bool = True):
        
        self.tokenizer = tokenizer
        self.data = dataframe
        self.question_len = question_len
        self.answers_len = answers_len
        self.question = self.data["question"]
        self.context = self.data["context"]
        self.switch = switch
        if self.switch:
            self.answers = self.data["answers"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):

        question = str(self.question[idx])
        context = str(self.context[idx])
        answers = str(self.answers[idx])

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            max_length=self.question_len,
            return_attention_mask=True,
            add_special_tokens=True,
            truncation="only_second",
        )
        
        samples = {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
        }

        if self.switch:
            targets = self.tokenizer.encode_plus(
                answers,
                max_length=self.answers_len,
                return_attention_mask=True,
                add_special_tokens=True,
                truncation=True,
            )
            target_ids = targets["input_ids"]
            samples["target_ids"] = target_ids
            samples["answers"] = answers
        return samples

In [9]:
# Динамический паддинг
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain

    def __call__(self, batch):
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target_ids"] = [sample["target_ids"] for sample in batch]
            output["answers"] = [sample["answers"] for sample in batch]
        # вычисляем максимальную длину токена в батче
        batch_max = max([len(ids) for ids in output["input_ids"]])

        # add padding
        if self.tokenizer.padding_side == "right":
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        
        # если тренируем то делаем тоже самое с таргетами
        if self.isTrain:
            batch_max = max([len(ids) for ids in output["target_ids"]])

            if self.tokenizer.padding_side == "right":
                output["target_ids"] = [s + (batch_max - len(s)) * [-100] for s in output["target_ids"]]
            else:
                output["target_ids"] = [(batch_max - len(s)) * [-100] + s for s in output["target_ids"]]
            output["target_ids"] = torch.tensor(output["target_ids"], dtype=torch.long)
        return output

# 2. Обучение

In [10]:
def train_epoch(tokenizer, model, device, loader, optimizer):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.apex)
    losses = []
    for i, data in enumerate(tqdm(loader), 0):
        
        inputs_ids = data["input_ids"].to(device, dtype=torch.long)
        inputs_mask = data["attention_mask"].to(device, dtype=torch.long)        
        target_ids = data["target_ids"].to(device, dtype=torch.long)
        
        with torch.cuda.amp.autocast(enabled=cfg.apex):
            outputs = model(
                input_ids=inputs_ids,
                attention_mask=inputs_mask,
                labels=target_ids,
            )

            loss = outputs.loss
        
        losses.append(loss.item())
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm)
        
        if i%cfg.n_accumulate==0:
            scaler.step(optimizer)
            scheduler.step()
            scaler.update()
            optimizer.zero_grad()
                    
    return np.mean(losses)

In [11]:
@torch.no_grad()
def validate(tokenizer, model, device, loader):
    losses = []
    
    model.eval()
     
    for _, data in enumerate(tqdm(loader), 0):

        inputs_ids = data["input_ids"].to(device, dtype=torch.long)
        inputs_mask = data["attention_mask"].to(device, dtype=torch.long)        
        target_ids = data["target_ids"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=inputs_ids,
            attention_mask=inputs_mask,
            labels=target_ids,
        )

        loss = outputs.loss
        losses.append(loss.item())

    return np.mean(losses)

In [12]:
transformers.logging.set_verbosity_error()

# tokenizer for encoding the text
tokenizer = T5Tokenizer.from_pretrained(cfg.model_name)
collate_fn = Collate(tokenizer)

model = T5ForConditionalGeneration.from_pretrained(cfg.model_name).to(device)

wandb.init(
      # Set the project where this run will be logged
      project="T5_FOR_QA_on_SQUAD_NU", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"test1")

# формируем тренировачный набор и лоадер
train_dataset = T5DataSet(
    train_data,
    tokenizer,
    cfg.source_max_len,
    cfg.target_max_len,
)

train_params = {
    "batch_size": cfg.batch_size,
    "shuffle": True,
    "num_workers": cfg.num_workers,
    "collate_fn": collate_fn,
    "pin_memory": True,
    "drop_last": True
}

train_loader = DataLoader(train_dataset, **train_params)

# формируем проверочный набор и лоадер
val_dataset = T5DataSet(
    val_data,
    tokenizer,
    cfg.source_max_len,
    cfg.target_max_len,
)

val_params = {
    "batch_size": cfg.batch_size * 2,
    "shuffle": False,
    "num_workers": cfg.num_workers,
    "collate_fn": collate_fn,
    "pin_memory": True,
    "drop_last": True
}

val_loader = DataLoader(val_dataset, **val_params)

# шедулер и оптимайзер
optimizer = AdamW(model.parameters(), lr=cfg.learning_rate, eps=cfg.eps, betas=cfg.betas)

num_train_steps = int(len(train_loader) / cfg.batch_size * cfg.epoch)
scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=cfg.num_warmup_steps,
        num_training_steps=num_train_steps
        )

best_loss = np.inf
for epoch in range(cfg.epoch):

    print(f'Epoch {epoch + 1}/{cfg.epoch}')
    print('-' * 10)
    
    train_loss = train_epoch(tokenizer, model, device, train_loader, optimizer)  
    print(f'Train loss {train_loss}')
    
    val_loss = validate(tokenizer, model, device, val_loader) 
    print(f'Val   loss {val_loss}\n')
    
    if val_loss < best_loss:
        # checkpoint
        model.save_pretrained(cfg.path_of_model)
        best_loss = val_loss
    wandb.log({"train_loss": train_loss, "val_loss": val_loss})
      
wandb.finish()

Epoch 1/3
----------


  0%|          | 0/11332 [00:00<?, ?it/s]

C:\Users\bitzh\anaconda3\envs\zhuk_home\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Train loss 1.5723296447318118


  0%|          | 0/629 [00:00<?, ?it/s]

...

  0%|          | 0/11332 [00:00<?, ?it/s]

Train loss 1.1802950676818575


  0%|          | 0/629 [00:00<?, ?it/s]

Val   loss 0.6631288986611632




train_loss,█▁▁
val_loss,▁▁▁
train_loss,1.1803
val_loss,0.66313


# 3. Prediction

In [7]:
def get_rouge(predictions: list, actual: list):
    """
    Calculates ROUGE metrics (RU too)
    :param predictions:
    :param labels:
    :return: Return dict of average of curent metrics
    """

    assert type(predictions) == list, "Hypotes not a list"
    assert type(actual) == list, "Targets not a list"

    i = 0
    for p in predictions:
        if p == '' or p == '.':
            predictions[i] = ' '
        i+=1
        
    rouge = Rouge()
    try:
        result = rouge.get_scores(predictions, actual, avg=True)
    except:
        print(predictions)
        print(actual)
        return
    return {k: {kk : round(vv, 2) for kk, vv in v.items()} for k, v in result.items()}

In [8]:
@torch.no_grad()
def predict(tokenizer, model, device, question, context, question_len):
    """
    Function to get models predictions
    """
    question = str(question)
    context = str(context)
    
    model.eval()

    inputs = tokenizer(
        question,
        context,
        max_length=question_len,
        return_attention_mask=True,
        add_special_tokens=True,
        truncation="only_second",
        padding="max_length",
        return_tensors="pt",
    )

    inputs_ids = inputs["input_ids"].to(device, dtype=torch.long)
    inputs_mask = inputs["attention_mask"].to(device, dtype=torch.long)        

    outputs = model.generate(
        input_ids=inputs_ids,
        attention_mask=inputs_mask,
        num_beams=2,  
        max_length=80,
        repetition_penalty=2.5,
        early_stopping=True,
        use_cache=True
    )

    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)
             for g in outputs][0]

    return preds

In [9]:
model = T5ForConditionalGeneration.from_pretrained(cfg.path_of_model).to(device)
tokenizer = T5Tokenizer.from_pretrained(cfg.model_name)

## 3.1. Val Data

In [11]:
idx = val_data.sample(1).index[0]
question = val_data.iloc[idx]['question']
context = val_data.iloc[idx]['context']
answer = val_data.iloc[idx]['answers']

pred = predict(tokenizer, model, device, question, context, cfg.source_max_len)
print(context + '\n', question + '\n', f'TRUE: {answer}\n', f'PRED: {pred}\n' )

В Одессе Бунин прожил почти полтора года — писал статьи для местных изданий, возглавлял литературный отдел газеты Южное слово , участвовал в деятельности основанного генералом Антоном Деникиным агентства ОСВАГ[91]. В частных разговорах он периодически упоминал о желании вступить в Добровольческую армию[92]. В интервью, данном газете Одесский листок (1918, № 120), писатель весьма резко отзывался о страшных контрастах эпохи — совпадении столетнего юбилея Тургенева с годовщиной революции[93][94]. Прозаик Иван Соколов-Микитов, общавшийся с Буниным в ту пору, рассказывал, что в Одессе Иван Алексеевич находился в крайне угнетённом состоянии[95].
 О чём говорил Бунин в интервью газете Одесский листок ?
 TRUE: о страшных контрастах эпохи
 PRED: О страшных контрастах эпохи



In [17]:
results = val_data.copy()
res = []
for q, c in tqdm(zip(val_data['question'], val_data['context']), total=len(val_data)):
    res.append(predict(tokenizer, model, device, q, c, cfg.source_max_len))
results['pred'] = res
results                

  0%|          | 0/5036 [00:00<?, ?it/s]

,context,question,answers,pred
0,Первые упоминания о строении человеческого тел...,Где встречаются первые упоминания о строении ч...,в Древнем Египте,Древнем Египте
1,Первые упоминания о строении человеческого тел...,Когда египетский врач Имхотеп впервые описал н...,В XXVII веке до н. э.,XXVII веке до н. э.
2,Телескоп имеет модульную структуру и содержит ...,Как называется корректирующая оптическая систе...,COSTAR,COSTAR
3,Критики теории Вегенера поставили во главу угл...,Какая теория была отвергнута после смерти Веге...,теория дрейфа материков,теория дрейфа материков
4,При нагревании кусочки янтаря становятся очень...,Чему не уступают по красоте изделия из прессов...,изделиям из монолитных камней,изделиям из монолитных камней
...,...,...,...,...
5031,Классическая трёхуровневая система накачки раб...,Где используется классическая трёхуровневая си...,в рубиновом лазере.,в рубиновом лазере
5032,Первая платёжная карта American Express появил...,какой инвестиционный банк входил в состав Amer...,Lehman Brothers,Lehman Brothers
5033,Следующий альбом Heroes был во многом созвучен...,С каким альбомом был созвучен альбом Дэвида Бо...,Low,Low
5034,"Одним из тех, на кого игра Данна произвела неи...",В каком техасском оркестре выступал гитарист Л...,Light Crust Doughboys,Light Crust Doughboys


In [11]:
get_rouge(results['pred'].tolist(), results['answer'].tolist())

{'rouge-1': {'r': 0.6, 'p': 0.59, 'f': 0.58},
 'rouge-2': {'r': 0.45, 'p': 0.45, 'f': 0.43},
 'rouge-l': {'r': 0.6, 'p': 0.59, 'f': 0.57}}

## 3.2. Train Data

In [18]:
idx = train_data.sample(1).index[0]
question = train_data.iloc[idx]['question']
context = train_data.iloc[idx]['context']
answer = train_data.iloc[idx]['answers']

pred = predict(tokenizer, model, device, question, context, cfg.source_max_len)
print(context + '\n', question + '\n', f'TRUE: {answer}\n', f'PRED: {pred}\n' )

В Сан-Марино существует республиканская форма правления. Главами государства являются два капитана-регента, назначаемые Большим Генеральным советом. Капитаны-регенты избираются на срок 6 месяцев, с 1 апреля до 1 октября и с 1 октября до 1 апреля каждого года. Они выполняют функции главы государства и осуществляют исполнительную власть. Большой Генеральный Совет является парламентом Республики, он состоит из 60 депутатов, избираемых всеобщим голосованием по системе пропорционального представительства сроком на 5 лет. Аренго, или ассамблея глав семейств, в древности было верховным органом, в настоящее время аренго сохранило за собой право модифицировать Статуты Республики и право петиции . Это последнее право используется и в наши дни — капитаны-регенты получают многочисленные прошения, предоставляемые гражданами в первое воскресенье после 1-го апреля и после 1-го октября. Поданные прошения в обязательном порядке должны быть рассмотрены в течение 6 месяцев.
 Какая форма правления в Сан-М

In [52]:
results_t = train_data.sample(2000).copy()
res = []
for q, c in tqdm(zip(results_t['question'], results_t['context']), total=len(results_t)):
    res.append(predict(tokenizer, model, device, q, c, cfg.source_max_len))
results_t['pred'] = res
results_t    

  0%|          | 0/2000 [00:00<?, ?it/s]

,context,question,answers,pred
3850,"Румы́нский язы́к (самоназвание — Limba română,...",Как еще именуется румынский язык?,В сравнительной лингвистике именуется также да...,Дако-румынским
32178,Артикль обеспечивает связность текста (дискурс...,В каком предложении последовательного повество...,первом,в первом предложении
41550,Во Франции пластика продолжала держаться парад...,"Кто прославился колоссальною статуей Свободы ,...",А. Бартольди,Вольтер
42711,Основные фонды относятся к производственным ак...,"Сколько должна быть минимальная цена объекта, ...",40 000 рублей,пятидесятикратной установленной законом минима...
29309,Большинство транспортных средств оснащены двиг...,какими двигателями оснащены большинство трансп...,двигателями внутреннего сгорания,двигателями внутреннего сгорания
...,...,...,...,...
25594,Классиками и отцами-основателями киберпанка на...,Когда жанр киберпанка набрал популярность?,В 90-е,90-е годы
24815,Во II веке до н. э. на Форуме стали всё больше...,Как диктатор Сулла назвал новую курию?,Корнелия,курия Корнелия
39867,"Благодаря реализации проектов СРП, Сахалинская...",Какая область находится на пятой строчке росси...,Сахалинская область,Сахалинская область
7609,"Химические остатки пива, датированные 3500—310...",Что получали строители египетских пирамид полу...,пиво,пиво


In [54]:
get_rouge(results_t['pred'].tolist(), results_t['answers'].tolist())

{'rouge-1': {'r': 0.66, 'p': 0.66, 'f': 0.64},
 'rouge-2': {'r': 0.47, 'p': 0.48, 'f': 0.46},
 'rouge-l': {'r': 0.66, 'p': 0.66, 'f': 0.64}}

# 4. Выводы

На текущий момент нет достойных метрик для определения качества обучения по данной задачи, теорритически им может стать рейтинг удолетворенности ответом в будущем. 

ель в первую очередь добится качественного повторения ответов из контекста, поэтому большинство ответов будет данно модели при обучение, но и с валидацией она справляется, хоть часть контекста и присуствует на обучение. 

Считаю текущий результат дойстойным и действенным. Дальнейшее улучшение возможно при увеличение выборки, сужении домена контекстов и замены модели на более бОльшую.